# Ruteo

Este notebook permite realizar la optimización de todos los pedidos de 1 día y realizar el proceso en detalle.

## Preparación de la Sesión

Cargamos a la sesión de Google Colab el repositorio con los datos y los módulos desarrollados. Este paso **no** es necesario si se corre el notebook de manera local.

In [1]:
!git clone https://github.com/JuanCruzC97/ope3-logistica.git &> /dev/null

In [2]:
cd ope3-logistica

/content/ope3-logistica


In [3]:
!pip install -e . &> /dev/null

## Módulos

Importamos librerías y módulos propios para llevar a cabo la optimización.

In [4]:
# Importamos las principales librerías de manejo de datos y visualización.
import pandas as pd
import numpy as np
import plotly.express as px

# Importamos el componente principal Ruteo.
from logistica.ruteo import Ruteo
from logistica.utils import load_inputs, preparar_df_pedidos

# Importamos el módulo creado con metaheuristicas.
import logistica.metaheuristicas as mh

## Carga de Datos

Usamos los datos de inputs en `data/inputs/data_inputs.xlsx` con el formato requerido para incorporar los datos de pedidos y camiones al modelo.

### Subir Datos Externos

En caso de trabajar en Google Colab y querer subir datos que no se encuentren en el repositorio se deben correr la siguientes 2 celdas. Si trabajamos localmente directamente podemos agregar los datos a la carpeta de inputs.

In [ ]:
# Solo correr esta celda si importamos datos externos de pedidos y camiones.
from google.colab import files
uploaded = files.upload()

Saving data_inputs.xlsx to data_inputs.xlsx


In [ ]:
!rm /content/ope3-logistica/data/inputs/data_inputs.xlsx
!mv data_inputs.xlsx /content/ope3-logistica/data/inputs/data_inputs.xlsx

## Generación de Datasets

Cargamos los datos del excel de inputs en la sesión de Google Colab en formato de dataframe. Usamos la función `load_inputs` con el nombre de la hoja que cargamos en su interior.

In [5]:
# Guardamos los datos de la hoja pedidos del input en un dataframe.
df_pedidos = load_inputs("pedidos")
df_pedidos

,cliente,pedidos1,pedidos2,pedidos3,pedidos4,pedidos5,pedidos6,coord_x,coord_y
0,A,4,3,5,2,6,6,0.257571,1.803726
1,B,6,6,5,4,1,3,1.523313,2.102301
2,C,1,6,6,2,6,7,0.710910,2.629800
3,D,8,6,5,2,8,2,1.013940,2.149740
4,E,4,1,2,4,2,3,0.609480,0.666810
5,F,3,1,6,8,6,5,0.998640,1.451790
6,G,3,2,3,7,4,3,1.434510,0.018180
7,H,5,8,5,2,4,8,2.171880,0.764910
8,I,5,2,3,4,2,6,1.412280,0.863190
9,J,1,8,6,8,1,2,2.386260,1.387260


In [6]:
# Guardamos los datos de la hoja camiones del input en un dataframe.
df_camiones = load_inputs("camiones")
df_camiones

,camion,carga_max,pedidos_max,dist_max
0,1,12,3,2
1,2,12,3,2
2,3,12,3,2
3,4,12,3,2
4,5,12,3,2
5,6,12,3,2


In [8]:
# Guardamos los datos de la hoja parámetros del input en un dataframe.
df_parametros_ruteo = load_inputs("parametros_ruteo")
df_parametros_ruteo

,costo_oportunidad,presupuesto
0,3000,1220


# Optimización Diaria

Nuevamente, este notebook permite realizar la asignación de todos los pedidos de 1 día en camiones y obtener una solución muy cercana o directamente óptima.

La siguiente celda permite elegir uno de los 6 pedidos disponibles en el dataset mediante un dropdown. Debemos elegir el pedido y correr dicha celda. La siguiente guarda en el objeto `df_pedido` la información del pedido particular y será uno de los inputs del modelo de Ruteo para realizar la optimización.

In [9]:
#@title Seleccionar Pedido
# Hola
pedidos = 'pedidos1' #@param ["pedidos1", "pedidos2", "pedidos3", "pedidos4", "pedidos5", "pedidos6"]

In [10]:
df_pedido = preparar_df_pedidos(df_pedidos, pedidos)
df_pedido

,cliente,pedidos,coord_x,coord_y
0,A,4,0.257571,1.803726
1,B,6,1.523313,2.102301
2,C,1,0.710910,2.629800
3,D,8,1.013940,2.149740
4,E,4,0.609480,0.666810
5,F,3,0.998640,1.451790
6,G,3,1.434510,0.018180
7,H,5,2.171880,0.764910
8,I,5,1.412280,0.863190
9,J,1,2.386260,1.387260


## Iniciamos el Ruteo

El siguiente paso es inciar el Ruteo. Al correr esta celda el objeto `ruteo` contiene toda la información de camiones y el pedido elegido a partir de los dataframes creados previamente.

Agregamos, usando el dataframe de parámetros creado a partir del input, los parámetros de costo de oportunidad y presupuesto.

In [12]:
ruteo = Ruteo(df_camiones, df_pedido, costo_oportunidad=df_parametros_ruteo.costo_oportunidad[0], presupuesto=df_parametros_ruteo.presupuesto[0])

Ahora debemos generar una solución incial. Esto se realiza con el método de `Ruteo` llamado `get_solucion_inicial`. Debemos definir como parámetros el modo de solución inicial y la semilla de generación de número aleatorios (puede ser `None`).

Los modos de generación de solución inicial es:
* `simple`: Método determinístico donde se busca asignar, en orden, todos los pedidos posibles a cada camión.
* `random`: Método aleatorio de generación de solución inicial.

In [13]:
#@title Parámetros Solución Inicial
mode = 'random' #@param ["simple", "random"]
random_state = 42 #@param {type:"number"}

In [14]:
# Generamos la solución inicial en la instancia ruteo y vemos un resumen del resultado.
ruteo.get_solucion_inicial(mode=mode, random_state=random_state)
ruteo.summary_ruteo()

,
Carga Total,47.00
Costo Camiones,55000.00
Costo Oportunidad,0.00
Costo Total,55000.00
Costo Total por tn,1170.21
Ahorro,-4.08


Teniendo la solución incial el siguiente paso es definir los parámetros de la optimización, realizada con el método de **recocido simulado**. 

* `Temperatura Inicial`: Valor inicial del parámetro temperatura.
* `Temperatura Final`: Valor final del parámetro temperatura.
* `k`: Cantidad de temperaturas a utilizar en el proceso (dentro del intervalo de temperatura incial y final).
* `Modo de Temperatura`: Permite elegir si la distribución de las `k` temperaturas en el intervalos (inicial y final) es lineal o no lineal.
* `Iteraciones`: Define la cantidad de iteraciones que realizamos por cada valor de temperatura diferente. Será la cantidad de vecinos generada por temperatura.
* `p`: Parámetro de probabilidad que modifica la generación de vecinos.
* `random_state`: Define la semilla de generación de valores aleatorios. Puede tomar valor `None` para no definirla.

In [15]:
#@title Parámetros Metaheurística
temp_inicial = 200 #@param {type:"number"}
temp_final = 1 #@param {type:"number"}
k = 150 #@param {type:"number"}
temp_mode = 'non_linear' #@param ["linear", "non_linear"]
iters = 25 #@param {type:"number"}
max_time = None #@param {type:"number"}
p = 1 #@param {type:"number"}
random_state = 42 #@param {type:"number"}

Del módulo de metaheurísticas usamos la función `sa` de recocido simulado. Devuelve dos objetos:

* `best_sol`: Es una instancia de la clase `Ruteo` que contiene la mejor solución encontrada. Tiene todos los pedidos y camiones en su interior con la mejor asignación encontrada.
* `history`: Es un diccionario que posee toda la información de cada iteración del proceso de optimización.

In [ ]:
# Corremos la optimización y mostramos el resumen de la mejor solución obtenida.
best_sol, history = mh.sa(ruteo, 
                          t_inicial=temp_inicial, 
                          t_final=temp_final, 
                          k=k, 
                          iters=iters, 
                          temp_mode=temp_mode, 
                          max_time=max_time,
                          prob=p, 
                          random_state=random_state)
print("\n")
best_sol.summary_ruteo(time=history.get("time"), iters=history.get("iters"))

In [ ]:
# Podemos ver el resumen de la asignación de los camiones con el siguiente método.
best_sol.summary_camiones()

,Cliente 1,Cliente 2,Cliente 3,Costo,Carga,Costo_tn,Ahorro
Camion 1,F,C,None,10000,10,1000.0,-18.03
Camion 2,B,None,None,5600,4,1400.0,14.75
Camion 3,G,E,None,11000,11,1000.0,-18.03
Camion 4,K,H,None,10000,10,1000.0,-18.03
Camion 5,A,I,L,11000,11,1000.0,-18.03
Camion 6,J,D,None,10000,10,1000.0,-18.03
Costo Oportunidad,,,,0,0,,
Total,,,,57600,56,1028.57,-15.69


In [ ]:
# Podemos ver el resumen de la asignación de pedidos con el siguiente método.
best_sol.summary_pedidos()

,Carga,Asignado,Camion
Pedido A,2,True,5
Pedido B,4,True,2
Pedido C,2,True,1
Pedido D,2,True,6
Pedido E,4,True,3
Pedido F,8,True,1
Pedido G,7,True,3
Pedido H,2,True,4
Pedido I,4,True,5
Pedido J,8,True,6


In [ ]:
# Podemos guardar los resultados de ruteo, camiones y pedidos en la carpeta de outputs.
best_sol.save_results("data/outputs/data_outputs.xlsx")

In [ ]:
# Podemos visualizar el proceso de optimización con el siguiente método.
mh.make_history_plots(history)

In [ ]:
# Podemos quedarnos con los datos del proceso de optimización con el siguiente método.
mh.get_history_df(history)

,temp,actual_sol,new_sol,best_sol,delta,p
0,99.849882,1336.45,1340.19,1336.45,-3.74,0.96
1,99.849882,1340.19,1336.45,1336.45,3.74,1.00
2,99.849882,1336.45,1321.50,1336.45,14.95,1.00
3,99.849882,1321.50,1321.50,1321.50,0.00,1.00
4,99.849882,1321.50,1304.67,1321.50,16.83,1.00
...,...,...,...,...,...,...
995,0.091105,1026.09,1026.09,1026.09,0.00,1.00
996,0.091105,1026.09,1034.78,1026.09,-8.69,0.00
997,0.091105,1026.09,1026.09,1026.09,0.00,1.00
998,0.091105,1026.09,1040.00,1026.09,-13.91,0.00


In [ ]:
# Podemos visualizar la solución inicial (en la instancia ruteo) y la mejor solución
# en la instancia (best_sol) con el siguiente método.
ruteo.plot_solution()
best_sol.plot_solution()